FIRST SECTION: CATEGORIZING CITIES AS HOT, MEDIUM, OR COLD

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlmodel import Session
from sqlmodel import SQLModel
import sys
import os
sys.path.append(os.path.abspath(".."))
from models.house import House
from db import engine

import os

# Key for changing state names to their abbreviations
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'Puerto Rico': 'PR', 'Virgin Islands': 'VI', 'District of Columbia':'DI', 'New Brunswick': 'NB',
    'Guam': 'GU'
}

# FIRST SECTION: CLASSIFYING TEMPERTATURES

In [ ]:
# Read data
tdf = pd.read_csv("../data/citytemperatures.csv")

# Rename CITY to City
tdf = tdf.rename(columns={"CITY": "City"})

# Convert month columns (2 to 13) to numeric
tdf.iloc[:, 2:14] = tdf.iloc[:, 2:14].apply(pd.to_numeric, errors="coerce")

# Calculate the average temperature across months
tdf["Temperature"] = tdf.iloc[:, 2:14].mean(axis=1)

# Compute the average temperature across multiple instances of the same city if there are multiple inputs (multiple rows for each city)
tdf_avg_temp = tdf.groupby('City', as_index=False)['Temperature'].mean()

# Drop duplicates
tdf = tdf.drop_duplicates(subset=['City'], keep='first')

# Calculate quantiles for splitting data into 3 parts
quantiles = tdf["Temperature"].quantile([0.33, 0.66])

# Function to classify temperature based on quantiles
def classify_temp(temp):
    if temp >= quantiles[0.66]:
        return "Hot"
    elif temp >= quantiles[0.33]:
        return "Medium"
    else:
        return "Cold"

# Apply classification
tdf["Category"] = tdf["Temperature"].apply(classify_temp)
temp_mapping = {'Cold': 0, 'Medium': 1, 'Hot': 2}
merged_df['Temperature'] = merged_df['Temperature'].map(temp_mapping)

# Select only 'City' and 'Category' columns for final (tdf final = tdff)
tdff = tdf[["City", "Category"]]

# Rename Category to Temperature
tdff = tdff.rename(columns={"Category": "Temperature"})

tdff

,City,Temperature
0,"BIRMINGHAM,AL",Hot
1,"HUNTSVILLE,AL",Medium
2,"MOBILE,AL",Hot
3,"MONTGOMERY,AL",Hot
4,"ANCHORAGE,AK",Cold
...,...,...
259,"POHNPEI- CAROLINE IS.,PC",Hot
260,"CHUUK- E. CAROLINE IS.,PC",Hot
261,"YAP- W CAROLINE IS.,PC",Hot
262,"SAN JUAN,PR",Hot


# SECOND SECTION: COMBINING THIS WITH HOUSING DATA

In [3]:
# Read first houying data
hdf = pd.read_csv("../data/housingdata1.csv")

# Drop rows where 'City' or 'State' is NaN
hdf = hdf.dropna(subset=['City', 'State'])

# revised housing dataframe
hdf = hdf[["State", "City", "Bedroom", "Bathroom", "Area", "ListedPrice"]]

# Read second housing dataset
hdf2 = pd.read_csv("../data/housingdata2.csv")

# Simplify second dataset
hdf2 = hdf2[["state", "city", "bed", "bath", "house_size", "price"]]

# Rename columns in second dataset to match the first one
hdf2.columns = ['State', 'City', 'Bedroom', 'Bathroom', 'Area', 'ListedPrice']

# Drop rows where 'City' or 'State' is NaN in hd2
hdf2 = hdf2.dropna(subset=['City', 'State'])

# Rename states to their abbreviations
hdf2['State'] = hdf2['State'].map(us_state_abbrev).fillna(hdf2['State'])

# Filter out all non-abbreviated places
hdf2 = hdf2[hdf2['State'].str.match(r'^[A-Z]{2}$', na=False)]

# Combining the two hdfs
hdf = pd.concat([hdf, hdf2], ignore_index=True)

# reformat housing data city to CITY,SI  (SI = STATE INITIAL)
hdf["City"] = hdf["City"] + "," + hdf["State"]

# Ensure key column (city) is of the same type and normalize it so they are all in the same format
hdf.loc[:, "City"] = hdf["City"].str.strip().str.lower()
tdff.loc[:, "City"] = tdff["City"].str.strip().str.lower()

# Merge datasets on 'city'
merged_df = hdf.merge(tdff[["City", "Temperature"]], on="City")

merged_df

,State,City,Bedroom,Bathroom,Area,ListedPrice,Temperature
0,AL,"huntsville,al",3.0,2.0,1225.0,250000.0,Medium
1,AL,"montgomery,al",3.0,2.0,1564.0,151000.0,Hot
2,AL,"mobile,al",3.0,3.0,2190.0,295000.0,Hot
3,AL,"huntsville,al",2.0,3.0,2099.0,199000.0,Medium
4,AL,"mobile,al",3.0,2.0,1194.0,150000.0,Hot
...,...,...,...,...,...,...,...
372288,WA,"walla walla,wa",2.0,1.0,1222.0,239000.0,Medium
372289,WA,"walla walla,wa",3.0,2.0,1450.0,445000.0,Medium
372290,WA,"walla walla,wa",3.0,2.0,2328.0,439000.0,Medium
372291,WA,"walla walla,wa",2.0,1.0,832.0,245000.0,Medium


# SECTION 2.5: IMPORTING IN OTHER STATEWISE TEMPERATURE DATA TO REPLACE NAN VALUES

In [ ]:
# Read state temperature data
stdf = pd.read_csv("../data/averagestatetemperatures.csv")[
    ["state", "average_temp"]
]

# Rename state to State
stdf = stdf.rename(columns={"state": "State"})

# Convert full state names to abbreviations
stdf['State'] = stdf['State'].map(us_state_abbrev)

# Compute the average temperature for each state if there are multiple rows with the same state
stdf_avg_temp = stdf.groupby('State', as_index=False)['average_temp'].mean()

# Drop duplicates
stdf = stdf.drop_duplicates(subset=['State'], keep='first')

# Calculate quantiles for splitting data into 3 parts
quantiles = stdf["average_temp"].quantile([0.33, 0.66])

# Function to classify temperature based on quantiles
def classify_temp(temp):
    if temp >= quantiles[0.66]:
        return "Hot"
    elif temp >= quantiles[0.33]:
        return "Medium"
    else:
        return "Cold"

# Apply classification
stdf["Temperature"] = stdf["average_temp"].apply(classify_temp)

# Remove avg temp
stdf = stdf.drop(columns=['average_temp'])

# Manually input values for states which were not in the state dataset
stdf.loc[len(stdf)] = ['ak', 'Cold']
stdf.loc[len(stdf)] = ['pr', 'Hot']
stdf.loc[len(stdf)] = ['di', 'Medium']
stdf.loc[len(stdf)] = ['vi', 'Hot']
stdf.loc[len(stdf)] = ['nb', 'Cold']
stdf.loc[len(stdf)] = ['hi', 'Hot']
stdf.loc[len(stdf)] = ['gu', 'Hot']

# Ensure data in State column is all in the same format
stdf.loc[:, "State"] = stdf["State"].str.strip().str.lower()
merged_df.loc[:, "State"] = merged_df["State"].str.strip().str.lower()

# Merge state temps into merged df, keeping BOTH temperature columns rather than combining state temps into the merged_df temps. This is because we only want to update
# merged_df temps if there was a NaN entry there
merged_df = merged_df.merge(stdf, on='State', suffixes=('_old', '_new'))

# Fill NaN values in the original Temperature column with values from the second dataset
merged_df['Temperature_old'] = merged_df['Temperature_old'].fillna(merged_df['Temperature_new'])

merged_df = merged_df.drop(columns=['Temperature_new'])

merged_df = merged_df.rename(columns={"Temperature_old": "Temperature"})

merged_df
merged_df.dtypes

State           object
City            object
Bedroom        float64
Bathroom       float64
Area           float64
ListedPrice    float64
Temperature     object
dtype: object

# THIRD SECTION: DOING THE SAME THING WITH QOL RATING

In [ ]:
# Read QOL data
qoldf = pd.read_csv("../data/qolcitydata.csv", encoding="ISO-8859-1")

# Clean city/state formatting
qoldf["LCITY"] = qoldf["LCITY"].str.strip().str.lower()
qoldf["LSTATE"] = qoldf["LSTATE"].str.strip().str.lower()
qoldf["LCITY"] = qoldf["LCITY"] + "," + qoldf["LSTATE"]
qoldf = qoldf.dropna(subset=["LCITY", "LSTATE"]).drop_duplicates(subset="LCITY")
qoldf = qoldf.rename(columns={"LCITY": "City"})

# Convert relevant columns to numeric safely
columns_to_convert = [
    "2022 Population",
    "2016 Crime Rate",
    "Unemployment",
    "AQI%Good",
    "WaterQualityVPV",
    "Cost of Living",
    "AVG C2I",
]

def safe_parse_float(x):
    try:
        if isinstance(x, str) and "/" in x:
            num, denom = x.strip().split("/")
            return float(num) / float(denom)
        return float(str(x).replace("$", "").replace("%", "").replace(",", "").strip())
    except:
        return np.nan


for col in columns_to_convert:
    qoldf[col] = qoldf[col].apply(safe_parse_float)

# Merge into merged_df
merged_df = merged_df.merge(qoldf[["Citwy"] + columns_to_convert], on="City")

merged_df

/var/folders/71/zyh8knb91j332nqms72qhc900000gn/T/ipykernel_54703/1111642847.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,9,11,12,13,14,15,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,37,38,39,40,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  qoldf = pd.read_csv("../data/qolcitydata.csv", encoding="ISO-8859-1")


,State,City,Bedroom,Bathroom,Area,ListedPrice,Temperature,2022 Population,2016 Crime Rate,Unemployment,AQI%Good,WaterQualityVPV,Cost of Living,AVG C2I
0,al,"huntsville,al",3.0,2.0,1225.0,250000.0,Medium,403565.0,0.037,2.18,80.94,0.0,76140.59,86.69
1,al,"montgomery,al",3.0,2.0,1564.0,151000.0,Hot,226361.0,0.047,3.17,80.94,1.0,74899.78,115.43
2,al,"mobile,al",3.0,3.0,2190.0,295000.0,Hot,411411.0,0.047,3.35,80.94,1.0,71947.38,115.28
3,al,"huntsville,al",2.0,3.0,2099.0,199000.0,Medium,403565.0,0.037,2.18,80.94,0.0,76140.59,86.69
4,al,"mobile,al",3.0,2.0,1194.0,150000.0,Hot,411411.0,0.047,3.35,80.94,1.0,71947.38,115.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350503,wa,"walla walla,wa",2.0,1.0,1222.0,239000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21
350504,wa,"walla walla,wa",3.0,2.0,1450.0,445000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21
350505,wa,"walla walla,wa",3.0,2.0,2328.0,439000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21
350506,wa,"walla walla,wa",2.0,1.0,832.0,245000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21


# FOURTH SECTION: FINALLY, ADD IN MEAN INCOME FOR EACH CITY

In [6]:
# Read data
idf = pd.read_csv("../data/meancityincome.csv", encoding="ISO-8859-1")[["State_ab", "City", "Mean"]]

# Rename columns
idf = idf.rename(columns={"State_ab": "State"})
idf = idf.rename(columns={"Mean": "MeanIncome"})

# Correct format of idf cities to match that of our merged_df
idf.loc[:, "City"] = idf["City"].str.strip().str.lower()
idf.loc[:, "State"] = idf["State"].str.strip().str.lower()
idf["City"] = idf["City"] + "," + idf["State"]

# Average out the mean and median entries for cases where one city has multiple entries
idf = idf.groupby('City')[['MeanIncome']].mean().round().reset_index()
idf = idf.drop_duplicates(subset='City')

# Merge with merged_df
merged_df = merged_df.merge(idf[["City", "MeanIncome"]], on="City")

nan_count = merged_df['Cost of Living'].isna().sum()
print(f"{nan_count}")

merged_df

0


,State,City,Bedroom,Bathroom,Area,ListedPrice,Temperature,2022 Population,2016 Crime Rate,Unemployment,AQI%Good,WaterQualityVPV,Cost of Living,AVG C2I,MeanIncome
0,al,"huntsville,al",3.0,2.0,1225.0,250000.0,Medium,403565.0,0.037,2.18,80.94,0.0,76140.59,86.69,63580.0
1,al,"montgomery,al",3.0,2.0,1564.0,151000.0,Hot,226361.0,0.047,3.17,80.94,1.0,74899.78,115.43,56275.0
2,al,"mobile,al",3.0,3.0,2190.0,295000.0,Hot,411411.0,0.047,3.35,80.94,1.0,71947.38,115.28,51465.0
3,al,"huntsville,al",2.0,3.0,2099.0,199000.0,Medium,403565.0,0.037,2.18,80.94,0.0,76140.59,86.69,63580.0
4,al,"mobile,al",3.0,2.0,1194.0,150000.0,Hot,411411.0,0.047,3.35,80.94,1.0,71947.38,115.28,51465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350424,wa,"walla walla,wa",2.0,1.0,1222.0,239000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21,65899.0
350425,wa,"walla walla,wa",3.0,2.0,1450.0,445000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21,65899.0
350426,wa,"walla walla,wa",3.0,2.0,2328.0,439000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21,65899.0
350427,wa,"walla walla,wa",2.0,1.0,832.0,245000.0,Medium,61890.0,0.034,4.84,92.89,0.0,76808.77,103.21,65899.0


# Section 5: Quality of Life State Data

In [7]:
qualityoflife = pd.read_csv("../data/qualityoflife.csv")

# Clean 'state' column and map to abbreviations
qualityoflife["state"] = qualityoflife["state"].str.strip()
qualityoflife["State"] = qualityoflife["state"].map(us_state_abbrev)

# Drop rows that failed to map (e.g., unrecognized state names)
qualityoflife = qualityoflife.dropna(subset=["State"])

# Lowercase for compatibility with merged_df
qualityoflife["State"] = qualityoflife["State"].str.lower()
merged_df["State"] = merged_df["State"].str.lower()

# Merge into merged_df on 'State'
merged_df = merged_df.merge(qualityoflife.drop(columns=["state"]), on="State")
merged_df


,State,City,Bedroom,Bathroom,Area,ListedPrice,Temperature,2022 Population,2016 Crime Rate,Unemployment,...,WaterQualityVPV,Cost of Living,AVG C2I,MeanIncome,QualityOfLifeTotalScore,QualityOfLifeQualityOfLife,QualityOfLifeAffordability,QualityOfLifeEconomy,QualityOfLifeEducationAndHealth,QualityOfLifeSafety
0,al,"huntsville,al",3.0,2.0,1225.0,250000.0,Medium,403565.0,0.037,2.18,...,0.0,76140.59,86.69,63580.0,45.61,40,1,40,48,32
1,al,"montgomery,al",3.0,2.0,1564.0,151000.0,Hot,226361.0,0.047,3.17,...,1.0,74899.78,115.43,56275.0,45.61,40,1,40,48,32
2,al,"mobile,al",3.0,3.0,2190.0,295000.0,Hot,411411.0,0.047,3.35,...,1.0,71947.38,115.28,51465.0,45.61,40,1,40,48,32
3,al,"huntsville,al",2.0,3.0,2099.0,199000.0,Medium,403565.0,0.037,2.18,...,0.0,76140.59,86.69,63580.0,45.61,40,1,40,48,32
4,al,"mobile,al",3.0,2.0,1194.0,150000.0,Hot,411411.0,0.047,3.35,...,1.0,71947.38,115.28,51465.0,45.61,40,1,40,48,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350424,wa,"walla walla,wa",2.0,1.0,1222.0,239000.0,Medium,61890.0,0.034,4.84,...,0.0,76808.77,103.21,65899.0,52.11,9,37,4,14,48
350425,wa,"walla walla,wa",3.0,2.0,1450.0,445000.0,Medium,61890.0,0.034,4.84,...,0.0,76808.77,103.21,65899.0,52.11,9,37,4,14,48
350426,wa,"walla walla,wa",3.0,2.0,2328.0,439000.0,Medium,61890.0,0.034,4.84,...,0.0,76808.77,103.21,65899.0,52.11,9,37,4,14,48
350427,wa,"walla walla,wa",2.0,1.0,832.0,245000.0,Medium,61890.0,0.034,4.84,...,0.0,76808.77,103.21,65899.0,52.11,9,37,4,14,48


# SECTION 6: UPLOADING TO SQL

In [8]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from supabase import create_client, Client
import os

# Load environment variables
load_dotenv()

# Retrieve database credentials from .env file
DB_URL = os.getenv("DATABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
DB_PASS = os.getenv("DATABASE_PASS")

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Fetch data from a table (replace 'your_table_name' with your actual table)
# response = supabase.table("Housing Data").select("*").execute()

# Convert to pandas DataFrame
# df = pd.DataFrame(response.data)

# print(df.head())

# Create SQLAlchemy engine
engine = create_engine(DB_URL, pool_size=5, max_overflow=10)

try:
    with engine.connect() as conn:
        print("Connected to the database successfully!")
except Exception as e:
    print(f"Error: {e}")

try:
    merged_df.to_sql('House', engine, if_exists='replace', index=False)
    print("Data uploaded successfully!")
except Exception as e:
    print(f"Error uploading data: {e}")

# Upload DataFrame to SQL table
# merged_df.to_sql("Housing Data", engine, if_exists="replace", index=False)

print("DataFrame successfully uploaded to the SQL database.")

Connected to the database successfully!
Data uploaded successfully!
DataFrame successfully uploaded to the SQL database.
